In [4]:
from halotools.mock_observables import rp_pi_tpcf, tpcf, s_mu_tpcf
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [5]:
def compute_real_tpcf(box, snapshot, r_bins, boxsize = 2000., num_threads = 1, m_min = 1.e12, m_max =1.e13):


    if(box == 15):
        filename = f'/raid/nbody/baorsd/run1{box:02d}/halo_catalog/R115_S{snapshot:03d}'
    else:
        filename = f'/raid/nbody/baorsd/run1{box:02d}/halo_catalog/S{snapshot:03d}_cen_rockstar'


    pos = np.fromfile(filename + '_pos.bin', dtype = np.float32)

    pos = np.reshape(pos, (-1, 3)) 

    m200c = np.fromfile(filename + '_mass.bin', dtype = np.float32)

    threshold = (m200c > m_min) & (m200c < m_max)

    pos = pos[threshold,:]
    
    real_tpcf = tpcf(pos, r_bins, period = boxsize, num_threads = num_threads)

    r_bins_c = 0.5*(r_bins[1:] + r_bins[:-1])

    return r_bins_c, real_tpcf



In [6]:
r_bins = np.logspace(-0.4, np.log10(150.), 100)


In [7]:
r_bins_c = 0.5*(r_bins[1:] + r_bins[:-1])

In [5]:
snapshot = 20
print(f'{snapshot:03d}')

020


In [9]:
n_boxes = 15

for box in range(1, n_boxes+1):

    print(f'Computing correlation function on box {box}')
    r_bins_c, real_tpcf = compute_real_tpcf(box, 11, r_bins, num_threads = 16)

    tp = {'r': r_bins_c, 'tpcf': real_tpcf}
    
    with open(f'/raid/c-cuesta/tpcfs/real_tpcf_box{box}_lowM_s011.pickle', 'wb') as fp:
        pickle.dump(tp, fp, protocol=pickle.HIGHEST_PROTOCOL)
        
    plt.loglog(tp['r'], tp['tpcf'], linestyle='', marker='o', markersize=2)
    #plt.xlim(0,2)
    plt.show()

Computing correlation function on box 1


KeyboardInterrupt: 

In [ ]:
def compute_redshift_tpcf(mode, box, r_bins, mu_bins = None, snapshot = 11,
                          m_min = 1.e12, m_max = 1.e13, nthreads = 16, 
                          los_direction=2, boxsize=2000.):

    lamda = 0.6844
    matter = 1. - lamda

    if(box == 15):
        filename = f'/raid/nbody/baorsd/run1{box:02d}/halo_catalog/R115_S{snapshot:03d}'
    else:
        filename = f'/raid/nbody/baorsd/run1{box:02d}/halo_catalog/S{snapshot:03d}_cen_rockstar'
    pos = np.fromfile(filename + '_pos.bin', dtype = np.float32)
    pos = np.reshape(pos, (-1, 3)) 

    vel = np.fromfile(filename + '_vel.bin', dtype = np.float32)
    vel = np.reshape(vel, (-1, 3)) 

    m200c = np.fromfile(filename + '_mass.bin', dtype = np.float32)

    threshold = (m200c > m_min) & (m200c < m_max)

    pos = pos[threshold,:]
    vel = vel[threshold,:]

    s_pos = pos.copy()
    
    if snapshot == 20:
        s_pos[:, los_direction] += vel[:, los_direction]/100. # to Mpc/h
    
    elif snapshot == 11:
        z = 0.484
        s_pos[:, los_direction] += vel[:, los_direction]/100. * np.sqrt((1 + z)/((1+z)**3*matter + lamda)) # to Mpc/h
        
    else:
        print('What is the redshift of that snapshot??')
        return None

    s_pos[:, los_direction][s_pos[:, los_direction]  > boxsize] -= boxsize

    s_pos[:, los_direction][s_pos[:, los_direction]  < 0.] += boxsize

    # rp_pi_tpcf always los = z
    if(los_direction != 2): 
        s_pos_old = s_pos.copy()
        s_pos[:,2] = s_pos_old[:,los_direction]
        s_pos[:,los_direction] = s_pos_old[:,2]
    
    r_bins_c = 0.5*(r_bins[1:] + r_bins[:-1])


    if mode == 'pi_sigma':
        tpcf = rp_pi_tpcf(s_pos, r_bins, r_bins, period=boxsize,
                estimator=u'Landy-Szalay', num_threads=nthreads)
        
        return r_bins_c, tpcf
    
    elif mode == 's_mu':
        tpcf = s_mu_tpcf(s_pos, r_bins, mu_bins, period=boxsize,
                estimator=u'Landy-Szalay', num_threads=nthreads)
        
        mu_bins_c = 0.5 * (mu_bins[1:] + mu_bins[:-1])
        
        return r_bins_c, mu_bins_c, tpcf
    
    else:
        print("Mode not valid!")
        return None


In [ ]:
r_bins = np.arange(0., 50., 0.5)
r_bins[0] = 0.0001
n_boxes = 15
n_mu_bins = 120

for box in range(1, n_boxes+1):
    
    mu_bins = np.linspace(0.,1.,n_mu_bins)


    print(f'Computing redshift space correlation function on box {box}')
    r_bins_c, tpcf_pi_sigma = compute_redshift_tpcf('pi_sigma',
                                                    box, r_bins)
    r_bins_c, mu_bins_c, tpcf_s_mu = compute_redshift_tpcf('s_mu', box,
                                                r_bins, mu_bins)

    tp = {'r': r_bins_c, 'mu_bins': mu_bins_c, 'pi_sigma': tpcf_pi_sigma, 's_mu': tpcf_s_mu}
    
    with open(f'/raid/c-cuesta/tpcfs/redshift_tpcf_box{box}_lowM_s011.pickle',
              'wb') as fp:
        
        pickle.dump(tp, fp, protocol=pickle.HIGHEST_PROTOCOL)